In [1]:
import os
import sys

parent_dir = os.path.abspath("/home/leandre/Projects/BioMoQA_Playground")

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.data_pipeline.ipbes.create_ipbes_raw import loading_pipeline_from_raw

In [2]:
datasets = loading_pipeline_from_raw(multi_label=True)

2025-07-18 12:25:20,031 - INFO - load raw positive datasets
2025-07-18 12:25:20,032 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-18 12:25:20,034 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-18 12:25:20,502 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-18 12:25:20,503 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-18 12:25:20,503 - INFO - 86


Resolving data files:   0%|          | 0/86 [00:00<?, ?it/s]

2025-07-18 12:25:20,666 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-18 12:25:20,666 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-18 12:25:20,667 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-18 12:25:20,824 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-18 12:25:20,825 - INFO - pos_ds features for IAS : {'DOI': Value(dtype='string', id=None), 'Title': Value(dtype='string', id=None), 'Abstract Note': Value(dtype='string', id=None), 'Language': Value(dtype='string', id=None), 'Item Type': Value(dtype='string', id=None), 'Publication Year': Value(dtype='int32', id=None)}
2025-07-18 12:25:20,825 - INFO - load raw negative dataset
2025-07-18 12:25:20,826 - INFO - creating corpus dataset


Resolving data files:   0%|          | 0/2524 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

2025-07-18 12:25:31,286 - INFO - dataset loaded
2025-07-18 12:25:31,287 - INFO - ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
2025-07-18 12:25:31,287 - INFO - renaming raw positive datasets
2025-07-18 12:25:31,302 - INFO - Finished positives and negatives creation pipeline


In [3]:
datasets

([Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 5251
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 6281
  }),
  Dataset({
      features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
      num_rows: 3109
  })],
 Dataset({
     features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
     num_rows: 4227131
 }))

In [4]:
pos_ds_list=datasets[0]
neg_ds=datasets[1]

In [5]:
clean_pos=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is None and batch['Item Type'][i] == 'journalArticle' and (batch['title'][i] is not None or batch['abstract'][i] is not None) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

In [6]:
print(clean_pos)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 328
})


In [7]:
clean_pos_a=pos_ds_list[0].filter(lambda batch : [batch['doi'][i] is not None and batch['Item Type'][i] == 'journalArticle' for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)

In [8]:
print(clean_pos_a)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 3829
})


In [9]:
clean_pos_b=clean_pos_a.filter(lambda batch : [(batch['abstract'][i] is not None and batch['title'][i] is not None) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(clean_pos_b)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 2253
})


In [10]:
none_abs=clean_pos_a.filter(lambda batch : [batch['abstract'][i] is None or batch['title'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 1576
})


In [11]:
print(none_abs[2])

{'doi': 'https://doi.org/10.1890/02-5002', 'title': 'Fire and grazing impacts on plant diversity and alien plant invasions in the southern Sierra Nevada', 'abstract': None, 'Item Type': 'journalArticle', 'Publication Year': 2003}


In [12]:
none_abs_instance=neg_ds.filter(lambda batch : [batch['display_name'][i] == 'Germination and early growth of Ailanthus< and tulip poplar in three levels of forest disturbance' if batch['display_name'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [13]:
pos_doi_set=set(pos_ds_list[0]['doi'])
pos_doi_set.remove(None)
none_abs_instance=neg_ds.filter(lambda batch : [any(batch['doi'][i].endswith(p_doi) for p_doi in pos_doi_set) if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=30)

In [14]:
print(len(pos_doi_set))

3861


In [15]:
print(none_abs_instance)

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 941
})


In [16]:
print(neg_ds['doi'][0:10])

['https://doi.org/10.1016/s0016-0032(29)90208-8', 'https://doi.org/10.1016/s0016-0032(29)90638-4', 'https://doi.org/10.1098/rstl.1829.0026', 'https://doi.org/10.1016/s0016-0032(29)90698-0', 'https://doi.org/10.1080/14786442908675166', 'https://doi.org/10.5962/bhl.title.109375', 'https://doi.org/10.1016/s0016-0032(29)90539-1', 'https://doi.org/10.1016/s0016-0032(29)90045-4', 'https://doi.org/10.1016/s0016-0032(29)90465-8', 'https://doi.org/10.1098/rstl.1816.0012']


In [17]:
print(neg_ds.filter(lambda batch : [batch['id'][i].endswith('W2142524821') if batch['doi'][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))

Dataset({
    features: ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr'],
    num_rows: 0
})


In [18]:
print(neg_ds['id'][0])

https://openalex.org/W1979751022


In [19]:
recent_pos=clean_pos_b.filter(lambda batch : [batch["Publication Year"][i]>=2021 if batch["Publication Year"][i] is not None else False for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20)
print(recent_pos)

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year'],
    num_rows: 158
})


In [ ]:
from src.data_pipeline.ipbes.fetch import fetch_crossref_metadata, get_metadata

params = {
    'mailto': 'leandre.catogni@hesge.ch'
}
filters = {
}
get_metadata(fetch_crossref_metadata('https://doi.org/10.1093/ee/nvaa108',params=params,filters=filters))



In [ ]:
from src.data_pipeline.ipbes.preprocess_ipbes import data_pipeline

clean_ds,folds_per_run=data_pipeline(2,1,balance_coeff=1,multi_label=True)
print(clean_ds)
print(clean_ds.features)

2025-07-18 12:25:44,549 - INFO - load raw positive datasets
2025-07-18 12:25:44,550 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-18 12:25:44,551 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-18 12:25:44,942 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES IAS_2352922
2025-07-18 12:25:44,942 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-18 12:25:44,943 - INFO - 86


Resolving data files:   0%|          | 0/86 [00:00<?, ?it/s]

2025-07-18 12:25:45,088 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES SUA_2344805
2025-07-18 12:25:45,089 - INFO - /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-18 12:25:45,090 - INFO - 128


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

2025-07-18 12:25:45,242 - INFO - Successfully loaded dataset from: /home/leandre/Projects/BioMoQA_Playground/data/IPBES/Raw/Positives/IPBES VA_2345372
2025-07-18 12:25:45,243 - INFO - pos_ds features for IAS : {'DOI': Value(dtype='string', id=None), 'Title': Value(dtype='string', id=None), 'Abstract Note': Value(dtype='string', id=None), 'Language': Value(dtype='string', id=None), 'Item Type': Value(dtype='string', id=None), 'Publication Year': Value(dtype='int32', id=None)}
2025-07-18 12:25:45,243 - INFO - load raw negative dataset
2025-07-18 12:25:45,244 - INFO - creating corpus dataset


Resolving data files:   0%|          | 0/2524 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

2025-07-18 12:25:46,714 - INFO - dataset loaded
2025-07-18 12:25:46,715 - INFO - ['id', 'display_name', 'author', 'ab', 'doi', 'topics', 'author_abbr']
2025-07-18 12:25:46,716 - INFO - renaming raw positive datasets
2025-07-18 12:25:46,733 - INFO - Finished positives and negatives creation pipeline
2025-07-18 12:25:46,733 - INFO - Filling missing metadata for positive datasets...
2025-07-18 12:25:46,734 - INFO - Starting metadata filling for positive datasets...
2025-07-18 12:25:46,734 - INFO - Processing IAS dataset with 5251 instances...
2025-07-18 12:25:46,734 - INFO - Starting metadata filling pipeline...
2025-07-18 12:25:47,152 - INFO - Found 1588 instances with missing title/abstract
2025-07-18 12:25:47,153 - INFO - Fetching metadata for 1586 unique DOIs...
2025-07-18 12:25:47,666 - INFO - Successfully fetched metadata for DOI: 10.1111/1749-4877.12606
2025-07-18 12:25:47,669 - INFO - Successfully fetched metadata for DOI: 10.5751/Es-04712-170134
2025-07-18 12:25:47,673 - INFO - S

In [ ]:

print(clean_ds.filter(lambda batch : [batch['doi'][i] is None and (batch['IAS']==1 or batch['SUA']==1 or batch['VA']==1 ) for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.filter(lambda batch : [batch['abstract'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.filter(lambda batch : [batch['title'][i] is None for i in range(len(batch['doi']))],batched=True,batch_size=1000,num_proc=20))
print(clean_ds.to_pandas()['Item Type'].unique())



Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 747444
})
Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})


Filter (num_proc=20):   0%|          | 0/4111303 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'title', 'abstract', 'Item Type', 'Publication Year', 'IAS', 'SUA', 'VA'],
    num_rows: 0
})


KeyboardInterrupt: 